# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# load libraire
!pip install ultralytics
from ultralytics import YOLO
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 22.1 MB/s eta 0:00:00


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
## Load the YOLO Model
model = YOLO("/content/drive/MyDrive/dataset/YOLO_Model.pt")

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
# Open video file
cap = cv2.VideoCapture("/content/drive/MyDrive/dataset/YOLO_Video.mp4")
assert cap.isOpened(), "Error reading video file"

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
# Get video properties
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
# street boundaries
top_street_y = int(0.1 * h)
bottom_street_y = int(0.9 * h)
left_street_x = int(0.1 * w)
right_street_x = int(0.9 * w)

# first>>>> start with zero vehicle counts
count_top = 0
count_bottom = 0
count_left = 0
count_right = 0

# Vehicle tracking
vehicle_positions = {}


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break


    Results = model(im0)

    # Check if Results is not None and has the expected structure
    if Results:
        # Process Results
        for result in Results:
            for detection in result.boxes:
                x1, y1, x2, y2 = map(int, detection.xyxy[0].tolist())
                label = int(detection.cls[0].item())
                conf = detection.conf[0].item()
            # calcalute centroid of the detected object
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

            # i classify objects based on their location
            if cy < top_street_y:
                count_top += 1
            elif cy > bottom_street_y:
                count_bottom += 1
            elif cx < left_street_x:
                count_left += 1
            elif cx > right_street_x:
                count_right += 1

            # Draw bounding box and label
            cv2.rectangle(im0, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(im0, f'{model.names[label]} {conf:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

             # Draw street lines and display counts
            cv2.line(im0, (0, top_street_y), (w, top_street_y), (0, 0, 255), 2)
            cv2.line(im0, (0, bottom_street_y), (w, bottom_street_y), (0, 255, 0), 2)
            cv2.line(im0, (left_street_x, 0), (left_street_x, h), (255, 0, 0), 2)
            cv2.line(im0, (right_street_x, 0), (right_street_x, h), (255, 255, 0), 2)


    # The number of vehicles is displayed on the left of the video.
    cv2.putText(im0, f'Top Street Exits: {count_top}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(im0, f'Bottom Street Exits: {count_bottom}', (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(im0, f'Left Street Exits: {count_left}', (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(im0, f'Right Street Exits: {count_right}', (10, 120),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
    cv2.putText(im0, 'SAMA', (w - 150, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    #  frame to output video
    video_writer.write(im0)

# Release resources
cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 5612.1ms
Speed: 21.6ms preprocess, 5612.1ms inference, 28.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 3262.1ms
Speed: 11.5ms preprocess, 3262.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2700.1ms
Speed: 3.8ms preprocess, 2700.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 92 cars, 2 trucks, 2 vans, 2668.5ms
Speed: 4.6ms preprocess, 2668.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 90 cars, 2 trucks, 2 vans, 2679.8ms
Speed: 4.4ms preprocess, 2679.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 91 cars, 2 trucks, 4 vans, 4392.6ms
Speed: 6.1ms preprocess, 4392.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 buss, 90 cars, 2 trucks, 4 vans, 2781.0ms
Speed: 5.4m

## Save and Submit Your Work

Submit both the notebook and the output video